In [7]:
import pandas as pd
from tqdm import tqdm_notebook as tqdm

def add_space(x):
    if (' ' not in x)&(len(x)<=5):
        return ' ' + x + ' '
    else:
        return x

def product_name_postprocess(x):
    x = x.replace('-',' ')
    x = x.strip()
    x = add_space(x)
    return x

# 讀取台塑網提供之(寶典人工手動修正過刪除線問題)
root = '../data/寶典/寶典人工處理後/'

df1 = pd.read_excel(root+'台塑企業_ 產品寶典20210303.xlsx',engine='openpyxl')[['公司代號','公司事業部門','品名']]

df2 = pd.read_excel(root+'寶典.v3.台塑網.20210901.xlsx',engine='openpyxl')[['CODIV','DIVNM','ITEMNM']]
df2 = df2.rename(columns={'ITEMNM':'品名','DIVNM':'公司事業部門','CODIV':'公司代號'})

df3 = pd.read_excel(root+'寶典.v4.20211001.xlsx',engine='openpyxl')[['CODIV','DIVNM','ITEMNM']]
df3 = df3.rename(columns={'ITEMNM':'品名','DIVNM':'公司事業部門','CODIV':'公司代號'})

df4 = pd.read_excel(root+'寶典.v5.20211006.xlsx',engine='openpyxl')[['CODIV','DIVNM','ITEMNM']]
df4 = df4.rename(columns={'ITEMNM':'品名','DIVNM':'公司事業部門','CODIV':'公司代號'})

df5 = pd.read_excel(root+'寶典.v6.20211020.xlsx',engine='openpyxl')[['CODIV','DIVNM','ITEMNM']]
df5 = df5.rename(columns={'ITEMNM':'品名','DIVNM':'公司事業部門','CODIV':'公司代號'})

df = df1.append(df2).append(df3).append(df4).append(df5) # 合併所有寶典
df['品名'] = df['品名'].apply(lambda x:product_name_postprocess(x)) #品名後處理

NAME = "GASOIL"

In [8]:
display(df[df['品名']==NAME])
display(df[df['品名']==NAME]['公司代號'].value_counts())

,公司代號,公司事業部門,品名
1060,60,國外處,GASOIL
330,60,國外處,GASOIL
330,60,國外處,GASOIL
333,60,國外處,GASOIL
1042,61,台塑石化,GASOIL


60    3
61    1
60    1
Name: 公司代號, dtype: int64

In [9]:
品名2部門 = {}
for i in tqdm(df['品名'].values):
    品名2部門[i] = df.loc[df['品名']==i,'公司事業部門'].values.tolist()
品名2代號 = {}
for i in tqdm(df['品名'].values):
    品名2代號[i] = df.loc[df['品名']==i,'公司代號'].values.tolist()

C:\Users\ricky\AppData\Local\Temp/ipykernel_12732/1830638321.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(df['品名'].values):


  0%|          | 0/5497 [00:00<?, ?it/s]

C:\Users\ricky\AppData\Local\Temp/ipykernel_12732/1830638321.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(df['品名'].values):


  0%|          | 0/5497 [00:00<?, ?it/s]

In [10]:
display(品名2部門[NAME])
display(品名2代號[NAME])

['國外處', '國外處', '國外處', '國外處', '台塑石化']

[60, '60', '60', '60', '61']

In [11]:
# 從traindata裡找
'''
train_df = pd.read_csv('../data/preprocess_for_SQUAD_產品.csv')
train_df['Y_label'] = train_df['Y_label'].apply(lambda x:x.strip()) #針對SPEC去除左右空白
display(train_df[(train_df['Y_label']==NAME)])
品名2代號訓練資料 = {}
for i in tqdm(train_df['Y_label'].values):
    品名2代號訓練資料[i] = train_df.dropna(subset=['EXPNO'],axis=0).loc[train_df['Y_label']==i,'EXPNO'].unique().tolist()
# 根據品名從訓練資料搜索EXPNO,然後把EXPNO代入保典裡找公司部門
def find_department(x):
    try:
        return [df.loc[df['公司代號']==品名2代號訓練資料[x],'公司事業部門'].value_counts().sort_values().index[-1]]
    except:
        return ['not from_pretrained']
display(品名2代號訓練資料[NAME])
display(find_department(NAME))
'''

"\ntrain_df = pd.read_csv('../data/preprocess_for_SQUAD_產品.csv')\ntrain_df['Y_label'] = train_df['Y_label'].apply(lambda x:x.strip()) #針對SPEC去除左右空白\ndisplay(train_df[(train_df['Y_label']==NAME)])\n品名2代號訓練資料 = {}\nfor i in tqdm(train_df['Y_label'].values):\n    品名2代號訓練資料[i] = train_df.dropna(subset=['EXPNO'],axis=0).loc[train_df['Y_label']==i,'EXPNO'].unique().tolist()\n# 根據品名從訓練資料搜索EXPNO,然後把EXPNO代入保典裡找公司部門\ndef find_department(x):\n    try:\n        return [df.loc[df['公司代號']==品名2代號訓練資料[x],'公司事業部門'].value_counts().sort_values().index[-1]]\n    except:\n        return ['not from_pretrained']\ndisplay(品名2代號訓練資料[NAME])\ndisplay(find_department(NAME))\n"

In [12]:
import numpy as np
'''
np.save('../data/對應表/品名2部門.npy',品名2部門)
np.save('../data/對應表/品名2代號.npy',品名2代號)
np.save('../data/對應表/品名2代號訓練資料.npy',品名2代號訓練資料)
'''

"\nnp.save('../data/對應表/品名2部門.npy',品名2部門)\nnp.save('../data/對應表/品名2代號.npy',品名2代號)\nnp.save('../data/對應表/品名2代號訓練資料.npy',品名2代號訓練資料)\n"